In [147]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
url = 'https://sosul.network/series/'

In [5]:
for p in range(1, 100):
    params = {'order_by' : 'review', 'page' : f'{p}'}
    res = requests.get(url, params = params)
    break

In [6]:
soup = BeautifulSoup(res.content, 'html.parser')
a_li = soup.find_all('div', class_='col-6 col-md-4 col-lg-15')

In [7]:
len(a_li)

30

In [150]:
def make_review(name, url):
    base_url = url
    df = pd.DataFrame(columns = ['name', 'score', 'review'])
    idx = 0
    while True:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'html.parser')
        review_li = soup.find_all('div', class_ = 'comment widget p-cb')
        for r in review_li:
            comment = r.find('div', class_ = 'text')
            score = comment.find('div', class_ = 'product-rate').find('star-rating')[':rating']
            try:
                r_text = comment.find('p', class_='content-txt').text
            except:
                r_text = comment.find('div', class_= 'ac-content').p.text
        
            df.loc[idx] = [name, score, r_text]
            idx += 1

        next_page = soup.find('ul', class_ = "pagination justify-content-center").find_all('li')
        p_num = next_page[-1].a['href']
        if p_num.split('=')[-1] == '#':
            break
        url = base_url + p_num

    return df

In [158]:
df = pd.DataFrame(columns = ['name', 'score', 'review'])
idx = 0
for i in a_li:
    a = i.find('div', class_='product-title')
    path = a.a['href']
    url = 'https://sosul.network' + path
    name = path.split('/')[-1]
    a = make_review(name, url)
    df = pd.concat([df, a])
    idx += 1
    print(len(a))
    if idx == 2:
        break

303
173


In [160]:
df

,name,score,review
0,190,0.5,내가하면 오리지널 남이하면 파쿠리라는 갓 마인드를 가진작가님의 작품거기에 문피아에 ...
1,190,2.0,"트위터 부녀자,페미코인을 제대로 탄 모범적인 소설그야말로 파쿠리의 집합체지만 다른소..."
2,190,1.0,대충 200화전까지는 별 4개쯤 줘도 괜찮다고 여기는데 연재가 계속될수록 기승전구원...
3,190,0.5,장갤평) 환생좌 짜집기 그럼에도 불구하고 내로남불 가득한 '그' 소설작가의 전작에서...
4,190,0.5,믿고 따라온 독자에게 똥물을 붓는 인성이 참 아름답다. 글이 어쩌고 하기 이전에 상...
...,...,...,...
168,3343,2.0,지겨움
169,3343,2.5,평범
170,3343,4.0,입문용으로 ㅅㅌㅊ임
171,3343,3.0,소설볼꺼면 웹툰보셈
